In [1]:
#Batch Normalisation : It is an algorithmic method which makes the training of Deep Neural Network faster and more stable.

In [2]:
#To get higher accuracy with less epoch.

In [3]:
#Add right before and right after the dense layer.

In [4]:
#Hyperparameter tuning using keras tuner

In [5]:
import pandas as pd
import numpy as np

df = pd.read_csv('https://raw.githubusercontent.com/npradaschnor/Pima-Indians-Diabetes-Dataset/master/diabetes.csv')

In [6]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [7]:
df.corr()['Outcome']

Pregnancies                 0.221898
Glucose                     0.466581
BloodPressure               0.065068
SkinThickness               0.074752
Insulin                     0.130548
BMI                         0.292695
DiabetesPedigreeFunction    0.173844
Age                         0.238356
Outcome                     1.000000
Name: Outcome, dtype: float64

In [8]:
X = df.iloc[:,:-1].values
y = df.iloc[:,-1].values

In [9]:
#scaling the data
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [10]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=1)

In [45]:
import tensorflow
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense,Dropout

In [12]:
model = Sequential()
model.add(Dense(32,activation='relu',input_dim=8))
model.add(Dense(1,activation='sigmoid'))

model.compile(optimizer='Adam',loss='binary_crossentropy',metrics=['accuracy'])
model.fit(X_train,y_train,batch_size=32,epochs=100,validation_data=(X_test,y_test))

Epoch 1/100
20/20 [==============================] - 5s 40ms/step - loss: 0.7241 - accuracy: 0.5065 - val_loss: 0.6896 - val_accuracy: 0.5649
Epoch 2/100
20/20 [==============================] - 0s 13ms/step - loss: 0.6492 - accuracy: 0.6075 - val_loss: 0.6279 - val_accuracy: 0.6753
Epoch 3/100
20/20 [==============================] - 0s 7ms/step - loss: 0.6002 - accuracy: 0.6824 - val_loss: 0.5875 - val_accuracy: 0.7338
Epoch 4/100
20/20 [==============================] - 0s 10ms/step - loss: 0.5681 - accuracy: 0.7215 - val_loss: 0.5610 - val_accuracy: 0.7662
Epoch 5/100
20/20 [==============================] - 0s 10ms/step - loss: 0.5452 - accuracy: 0.7329 - val_loss: 0.5407 - val_accuracy: 0.7922
Epoch 6/100
20/20 [==============================] - 0s 8ms/step - loss: 0.5276 - accuracy: 0.7410 - val_loss: 0.5248 - val_accuracy: 0.7987
Epoch 7/100
20/20 [==============================] - 0s 12ms/step - loss: 0.5146 - accuracy: 0.7655 - val_loss: 0.5128 - val_accuracy: 0.7987
Epoch 8/

In [13]:
#How to select appropriate optimizer
#No. of nodes (neurons) in a layer
#No. of hidden layer
#All in one model

In [14]:
#1. #How to select appropriate optimizer
!pip install -q -U keras-tuner
import keras_tuner as kt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 1.4 MB/s eta 0:00:00


In [15]:
#Now we have to build a function
def build_model(hp):
  model = Sequential()
  model.add(Dense(32,activation = 'relu',input_dim=8))
  model.add(Dense(1,activation='sigmoid'))

  optimizer = hp.Choice('optimizer',values=['adam','sgd','rmsprop','adadelta','nadam'])

  model.compile(optimizer=optimizer,loss='binary_crossentropy',metrics=['accuracy'])

  return model

In [16]:
tuner = kt.RandomSearch(build_model,
                        objective='val_accuracy',
                        max_trials=5)

In [17]:
tuner.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

Trial 4 Complete [00h 00m 04s]
val_accuracy: 0.7532467246055603

Best val_accuracy So Far: 0.8051947951316833
Total elapsed time: 00h 00m 16s


In [18]:
models = tuner.get_best_models(num_models=2)
best_model = models[0]
best_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 32)                288       
                                                                 
 dense_1 (Dense)             (None, 1)                 33        
                                                                 
Total params: 321 (1.25 KB)
Trainable params: 321 (1.25 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [19]:
tuner.results_summary()

Results summary
Results in ./untitled_project
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 2 summary
Hyperparameters:
optimizer: nadam
Score: 0.8051947951316833

Trial 0 summary
Hyperparameters:
optimizer: adam
Score: 0.7727272510528564

Trial 3 summary
Hyperparameters:
optimizer: rmsprop
Score: 0.7532467246055603

Trial 1 summary
Hyperparameters:
optimizer: adadelta
Score: 0.48051947355270386


In [20]:
tuner.get_best_hyperparameters()[0].values

{'optimizer': 'nadam'}

In [21]:
model = tuner.get_best_models(num_models=1)[0]
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 32)                288       
                                                                 
 dense_1 (Dense)             (None, 1)                 33        
                                                                 
Total params: 321 (1.25 KB)
Trainable params: 321 (1.25 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [22]:
model.fit(X_train,y_train,batch_size=32,epochs=100,initial_epoch=6,validation_data=(X_test,y_test))

Epoch 7/100
20/20 [==============================] - 1s 18ms/step - loss: 0.5499 - accuracy: 0.7215 - val_loss: 0.5255 - val_accuracy: 0.8182
Epoch 8/100
20/20 [==============================] - 0s 6ms/step - loss: 0.5319 - accuracy: 0.7313 - val_loss: 0.5084 - val_accuracy: 0.8182
Epoch 9/100
20/20 [==============================] - 0s 5ms/step - loss: 0.5175 - accuracy: 0.7345 - val_loss: 0.4953 - val_accuracy: 0.8117
Epoch 10/100
20/20 [==============================] - 0s 6ms/step - loss: 0.5063 - accuracy: 0.7508 - val_loss: 0.4858 - val_accuracy: 0.8052
Epoch 11/100
20/20 [==============================] - 0s 6ms/step - loss: 0.4977 - accuracy: 0.7541 - val_loss: 0.4779 - val_accuracy: 0.8052
Epoch 12/100
20/20 [==============================] - 0s 6ms/step - loss: 0.4898 - accuracy: 0.7606 - val_loss: 0.4708 - val_accuracy: 0.8052
Epoch 13/100
20/20 [==============================] - 0s 6ms/step - loss: 0.4834 - accuracy: 0.7671 - val_loss: 0.4657 - val_accuracy: 0.8117
Epoch 14

In [23]:
#Finding out the right numbers of neuron in the layer using keras tuner

def build_model(hp):

  model = Sequential()
  units = hp.Int('units',min_value=8,max_value=128)
  model.add(Dense(units=units, activation='relu',input_dim=8))
  model.add(Dense(1,activation='sigmoid'))

  model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])

  return model

In [24]:
tuner = kt.RandomSearch(build_model,
                        objective='val_accuracy',
                        max_trials=5,directory='mydir',
                        project_name='srk')

In [25]:
tuner.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

Trial 5 Complete [00h 00m 02s]
val_accuracy: 0.7922077775001526

Best val_accuracy So Far: 0.798701286315918
Total elapsed time: 00h 00m 11s


In [26]:
tuner.search_space_summary()

Search space summary
Default search space size: 1
units (Int)
{'default': None, 'conditions': [], 'min_value': 8, 'max_value': 128, 'step': 1, 'sampling': 'linear'}


In [27]:
tuner.results_summary()

Results summary
Results in mydir/srk
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 3 summary
Hyperparameters:
units: 116
Score: 0.798701286315918

Trial 2 summary
Hyperparameters:
units: 95
Score: 0.7922077775001526

Trial 4 summary
Hyperparameters:
units: 13
Score: 0.7922077775001526

Trial 1 summary
Hyperparameters:
units: 37
Score: 0.7857142686843872

Trial 0 summary
Hyperparameters:
units: 111
Score: 0.7662337422370911


In [28]:
tuner.get_best_hyperparameters()[0].values

{'units': 116}

In [29]:
model = tuner.get_best_models(num_models=1)[0]
model.fit(X_train,y_train,batch_size=32,epochs=32,initial_epoch=6,validation_data=(X_test,y_test))

Epoch 7/32
20/20 [==============================] - 1s 13ms/step - loss: 0.5393 - accuracy: 0.7508 - val_loss: 0.5129 - val_accuracy: 0.8052
Epoch 8/32
20/20 [==============================] - 0s 6ms/step - loss: 0.5084 - accuracy: 0.7606 - val_loss: 0.4893 - val_accuracy: 0.7987
Epoch 9/32
20/20 [==============================] - 0s 5ms/step - loss: 0.4894 - accuracy: 0.7671 - val_loss: 0.4765 - val_accuracy: 0.7922
Epoch 10/32
20/20 [==============================] - 0s 5ms/step - loss: 0.4769 - accuracy: 0.7687 - val_loss: 0.4692 - val_accuracy: 0.7987
Epoch 11/32
20/20 [==============================] - 0s 5ms/step - loss: 0.4690 - accuracy: 0.7671 - val_loss: 0.4661 - val_accuracy: 0.8182
Epoch 12/32
20/20 [==============================] - 0s 5ms/step - loss: 0.4628 - accuracy: 0.7818 - val_loss: 0.4630 - val_accuracy: 0.8052
Epoch 13/32
20/20 [==============================] - 0s 5ms/step - loss: 0.4586 - accuracy: 0.7801 - val_loss: 0.4607 - val_accuracy: 0.7987
Epoch 14/32
20/

In [30]:
#creating no of dense layers

def build_model(hp):

  model = Sequential()

  model.add(Dense(83, activation='relu',input_dim=8))

  for i in range(hp.Int('num_layers',min_value=1,max_value=10)):

    model.add(Dense(72,activation='relu'))

  model.add(Dense(1,activation='sigmoid'))

  model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])

  return model


In [31]:
tuner = kt.RandomSearch(build_model,
                        max_trials=3,
                        directory='mydir',
                        project_name='num_layers',
                        objective='val_accuracy')

In [33]:
tuner.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

Trial 3 Complete [00h 00m 02s]
val_accuracy: 0.8051947951316833

Best val_accuracy So Far: 0.8246753215789795
Total elapsed time: 00h 00m 11s


In [34]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 10}

In [35]:
model = tuner.get_best_models(num_models=1)[0]

In [36]:
model.fit(X_train,y_train,batch_size=32,epochs=100,initial_epoch=6)

Epoch 7/100
20/20 [==============================] - 2s 6ms/step - loss: 0.4762 - accuracy: 0.7443
Epoch 8/100
20/20 [==============================] - 0s 6ms/step - loss: 0.4621 - accuracy: 0.7736
Epoch 9/100
20/20 [==============================] - 0s 6ms/step - loss: 0.4490 - accuracy: 0.7834
Epoch 10/100
20/20 [==============================] - 0s 6ms/step - loss: 0.4333 - accuracy: 0.7932
Epoch 11/100
20/20 [==============================] - 0s 6ms/step - loss: 0.4297 - accuracy: 0.8062
Epoch 12/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4195 - accuracy: 0.7980
Epoch 13/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4187 - accuracy: 0.7997
Epoch 14/100
20/20 [==============================] - 0s 5ms/step - loss: 0.3959 - accuracy: 0.8241
Epoch 15/100
20/20 [==============================] - 0s 5ms/step - loss: 0.3951 - accuracy: 0.8241
Epoch 16/100
20/20 [==============================] - 0s 6ms/step - loss: 0.3589 - accuracy: 0.8404
Epo

In [37]:
tuner.search_space_summary()

Search space summary
Default search space size: 1
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 1, 'max_value': 10, 'step': 1, 'sampling': 'linear'}


In [38]:
tuner.results_summary()

Results summary
Results in mydir/num_layers
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 1 summary
Hyperparameters:
num_layers: 10
Score: 0.8246753215789795

Trial 2 summary
Hyperparameters:
num_layers: 3
Score: 0.8051947951316833

Trial 0 summary
Hyperparameters:
num_layers: 6
Score: 0.7792207598686218


In [46]:
#Multiple Parameters
def build_model(hp):

  model = Sequential()

  counter = 0
  for i in range(hp.Int('num_layers',min_value=1,max_value=10)):

    if counter==0:
      model.add(
          Dense(hp.Int('units'+str(i),min_value=8,max_value=128,step=8),
                activation=hp.Choice('activation'+str(i),values=['relu','tanh','sigmoid']),
                input_dim=8
                ))
      model.add(Dropout(hp.Choice('dropout'+str(i),values=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])))
    else:
      model.add(
          Dense(hp.Int('units'+str(i),min_value=8,max_value=128,step=8),
                activation=hp.Choice('activation'+str(i),values=['relu','tanh','sigmoid'])
                ))

    counter+=1

  model.add(Dense(1,activation='sigmoid'))
  model.compile(optimizer=hp.Choice('optimizer',values=['rmsprop','adam','sgd','nadam','adadelta']),
                loss='binary_crossentropy',
                metrics=['accuracy'])

  return model


In [47]:
tuner = kt.RandomSearch(build_model,
                        objective='val_accuracy',
                        max_trials=3,
                        directory='mydir',
                        project_name='final1')

In [48]:
tuner.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

Trial 3 Complete [00h 00m 07s]
val_accuracy: 0.8246753215789795

Best val_accuracy So Far: 0.8246753215789795
Total elapsed time: 00h 00m 22s


In [49]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 9,
 'units0': 96,
 'activation0': 'tanh',
 'dropout0': 0.2,
 'optimizer': 'adam',
 'units1': 56,
 'activation1': 'sigmoid',
 'units2': 32,
 'activation2': 'tanh',
 'units3': 24,
 'activation3': 'sigmoid',
 'units4': 128,
 'activation4': 'relu',
 'units5': 40,
 'activation5': 'tanh',
 'units6': 16,
 'activation6': 'tanh',
 'units7': 88,
 'activation7': 'relu',
 'units8': 72,
 'activation8': 'sigmoid'}

In [50]:
tuner.results_summary()

Results summary
Results in mydir/final1
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 2 summary
Hyperparameters:
num_layers: 9
units0: 96
activation0: tanh
dropout0: 0.2
optimizer: adam
units1: 56
activation1: sigmoid
units2: 32
activation2: tanh
units3: 24
activation3: sigmoid
units4: 128
activation4: relu
units5: 40
activation5: tanh
units6: 16
activation6: tanh
units7: 88
activation7: relu
units8: 72
activation8: sigmoid
Score: 0.8246753215789795

Trial 1 summary
Hyperparameters:
num_layers: 9
units0: 120
activation0: tanh
dropout0: 0.7
optimizer: nadam
units1: 40
activation1: tanh
units2: 64
activation2: relu
units3: 8
activation3: relu
units4: 8
activation4: relu
units5: 8
activation5: relu
units6: 8
activation6: relu
units7: 8
activation7: relu
units8: 8
activation8: relu
Score: 0.798701286315918

Trial 0 summary
Hyperparameters:
num_layers: 3
units0: 64
activation0: sigmoid
dropout0: 0.6
optimizer: adadelta
units1: 8
activation1: relu
units2: 8
ac

In [51]:
model = tuner.get_best_models(num_models=1)[0]
model.fit(X_train,y_train,epochs=200,initial_epoch=5,validation_data=(X_train,y_train))

Epoch 6/200
20/20 [==============================] - 4s 34ms/step - loss: 0.4901 - accuracy: 0.7720 - val_loss: 0.4740 - val_accuracy: 0.7752
Epoch 7/200
20/20 [==============================] - 0s 9ms/step - loss: 0.4793 - accuracy: 0.7638 - val_loss: 0.4938 - val_accuracy: 0.7541
Epoch 8/200
20/20 [==============================] - 0s 9ms/step - loss: 0.4871 - accuracy: 0.7671 - val_loss: 0.4739 - val_accuracy: 0.7557
Epoch 9/200
20/20 [==============================] - 0s 10ms/step - loss: 0.4949 - accuracy: 0.7638 - val_loss: 0.4691 - val_accuracy: 0.7704
Epoch 10/200
20/20 [==============================] - 0s 8ms/step - loss: 0.5079 - accuracy: 0.7590 - val_loss: 0.4780 - val_accuracy: 0.7752
Epoch 11/200
20/20 [==============================] - 0s 11ms/step - loss: 0.4748 - accuracy: 0.7590 - val_loss: 0.4666 - val_accuracy: 0.7720
Epoch 12/200
20/20 [==============================] - 0s 10ms/step - loss: 0.4701 - accuracy: 0.7785 - val_loss: 0.4752 - val_accuracy: 0.7638
Epoch 